## Fast Eective Rule Induction
https://www.semanticscholar.org/paper/Fast-Effective-Rule-Induction-Cohen/6665e03447f989c9bdb3432d93e89b516b9d18a7

#### IREP
Given a training set $\mathcal{D}$ with binary labels and all possible features $\mathcal{F}$ in $\mathcal{F}$,
1. Randomly choose 2/3 from $\mathcal{D}$ as the growing set $\mathcal{G}$ and the rest 1/3 becomes the pruning set $\mathcal{P}$. 
2. Initialize an empty rule set $\mathcal{R}$.
3. While there are still positive instances in $\mathcal{G}$:
    1. Initialize a empty rule $R$.
    2. Until all instances that satisfy $R$ are positive (accuracy of $R$ is 1) or there is no feature to add: 
        1. For every feature $f \in \mathcal{F}$ not in $R$ and every possible value $v$ of $f$:
            1. Create a temp rule $R_{t}$ by copying current $R$. 
            2. Add $f=v$ to $R_{t}$.
            3. Calculate FOIL's information gain of $R_{t}$: $\mathrm{Foil}(R, R_{t})$ based on $\mathcal{G}$.
        2. Get the $R_{t}^{max}$ with the max value of $\mathrm{Foil}(R_{t})$.
        3. $R=R_{t}^{max}$
    3. For all $(f=v)_i \in R$ starting from the last added one to the first one:
        1. Get $R_{p}$ by removing $(f=v)_i$ from $R$.
        2. If $\mathrm{Value}(R_{p}) \geq \mathrm{Value}(R)$: then $R=R_{p}$
    4. If the accuracy of $R < 0.5$ on $\mathcal{P}$: break
    5. Add $R$ to $\mathcal{R}$.
    6. Remove instances that are covered by $R$ from $\mathcal{G}$.
4. Return $\mathcal{R}$

**[FOIL's information gain]**: it gives how much information entropy is reduced from $R_{old}$ to $R_{new}$.
$$ \mathrm{Foil}(R_{old}, R_{new}) = P(R_{old}) (\log_{2}(\frac{P(R_{old})}{P(R_{old}) + N(R_{old})}) - \log_{2}(\frac{P(R_{new})}{P(R_{new}) + N(R_{new})})) $$
where $P(R)$ is the number of positive instances covered by $R$ and $N(R)$ is the number of negative instances covered by $R$.

**[IREP Rule value]**: 
$$ \mathrm{Value}(R) = \frac{P(R) + (N - N(R))}{P + N} $$
where $P$ is the total number of positive instances,  $N$ is the total number of negative instances, $P(R)$ is the number of positive instances covered by $R$ and $N(R)$ is the number of negative instances covered by $R$.

#### RIPPER
RIPPER is an improved version of IREP and the changes are:
1. Use a different equation (RIPPER Rule value) to replace IREP's rule value equation.
2. Replace the stopping condition (line 3.D. in the pseudocode above) with the following logic on the dataset $\mathcal{P}$ ($d$ is a hyperparameter with the default value of 64):
    1. Calculate the *total description length* of $R$: $\mathrm{MDL}(R)$.
    2. If $\mathrm{MDL}(R) > \mathrm{MDL_{min}} + d$: break
    3. If $\mathrm{MDL}(R) < \mathrm{MDL_{min}}$: $\mathrm{MDL_{min} = MDL}(R)$
3. The version of IREP with the 2 changes above is called IREP*. After we get a rule set $\mathcal{R}$ from one run of IREP*, we perform the followings on the dataset $\mathcal{P}$:
    1. For each $R_{i} \in \mathcal{R}$:
        1. Get the *replacement* $\hat{R}_{i}$ of $R_{i}$ by growing from an empty rule until all instance that satisfy $\hat{R}_{i}$ are positive. 
        2. Pruning $\hat{R}_{i}$ by minimizing error of the entire rule set $\mathcal{\hat{R}} = R_{1}, ..., \hat{R}_{i}, ..., R_{n}$ (no $R_i$) on $\mathcal{P}$.
        3. Get the *revision* $\bar{R}_{i}$ of $R_{i}$ by growing from $R_{i}$ until all instance that satisfy $\bar{R}_{i}$ are positive.
        4. Pruning $\bar{R}_{i}$ by minimizing error of the entire rule set $\mathcal{\bar{R}} = R_{1}, ..., \bar{R}_{i}, ..., R_{n}$ (no $R_i$) on $\mathcal{P}$.
        5. $\mathcal{R} = \underset{\mathcal{R_i} \in \{ \mathcal{R}, \hat{\mathcal{R}}, \bar{\mathcal{R}} \}}{\mathrm{argmin}} \sum_{R_{i} \in \mathcal{R_{i}}} \mathrm{MDL}(R_i)$
4. Run IREP* again to cover remaining positive instances. 

**[RIPPER Rule value]**: 
$$ \mathrm{Value}(R) = \frac{P(R) - N(R)}{P(R) + N(R)} $$
where $P(R)$ is the number of positive instances covered by $R$ and $N(R)$ is the number of negative instances covered by $R$

**[Total description length]**: MDL is composed of model description length and exceptions description length. 
$$ \mathrm{MDL}(R) = \mathrm{MDL_{M}}(R) + \mathrm{MDL_{E}}(R) $$
Model description length evaluates the rule itself:
$$ \mathrm{MDL_{M}}(R) = 0.5(k\log_{2}\frac{1}{r} + (n-k) \log_2\frac{1}{1-r} + \lVert k \rVert) $$
where $k$ is the number of features in the rule, $n$ is the number of all features, and $r = \frac{k}{n}$. $\lVert k \rVert = \log_{2}(k)$. The 0.5 factor is to account for possible redundancies. \
Exceptions description length evaluates the performance of the rule on the dataset:
$$ \mathrm{MDL_{E}}(R) = \log_{2}{P(R) \choose \mathit{FP}(R)} + \log_{2}{N(R) \choose \mathit{FN}(R)}$$ 
where $P(R)$ is the number of positive instances covered by $R$, $N(R)$ is the number of negative instances covered by $R$ $\mathit{FP}(R)$ is the number of false positives covered by $R$, $\mathit{FN}(R)$ is the number of false negatives covered by $R$ 